# Part 1

In [473]:
with open('day19-rules.txt') as f:
    txt = f.read().splitlines()
    
txt[:10]

['136: 83 66 | 102 116',
 '120: 83 66 | 58 116',
 '26: 56 116 | 105 66',
 '42: 37 66 | 100 116',
 '103: 116 33 | 66 47',
 '35: 116 21 | 66 105',
 '82: 113 116 | 93 66',
 '96: 66 60 | 116 61',
 '25: 116 68 | 66 34',
 '4: 116 66']

In [474]:
import re

rules = {n:r for (n,r) in [re.findall("(\d+): (.*)", r)[0] for r in txt]}
rules['25']

'116 68 | 66 34'

In [475]:
'116 116 | 116 66'.split(' ')

['116', '116', '|', '116', '66']

In [476]:
def build_rule( r, rules ):
    
    m = re.match("\"([ab])\"", r)
    if m:
        return m[1]

    subrules = r.split(' ')
    
    rule = '('
    
    for sr in subrules:
        if re.match("\d+", sr):
            rule += build_rule( rules[sr], rules )
        elif sr == '|':
            rule += '|'
            
    rule += ')'
    
    return rule

In [477]:
build_rule('26', rules)

'(((ba|aa)a|(aa)b))'

In [478]:
with open('day19-messages.txt') as f:
    msgs = f.read().splitlines()

In [479]:
r = build_rule( '0', rules )
sum(1 if re.fullmatch(r, m) else 0 for m in msgs)

269

# Part 2

In [522]:
rules['8'] = '42 | 42 8'
rules['11'] = '42 31 | 42 11 31'

In [523]:
def build_rule( idx, rules ):
    
    r = rules[idx]
    
    m = re.match("\"([ab])\"", r)
    if m:
        return m[1]

    subrules = r.split(' ')
    
    rule = '(?:'
    recursive = False
    
    for sr in subrules:
        if re.match("\d+", sr):
            if sr == idx:
                rule += '+'
                recursive = True
            else:
                rule += build_rule( sr, rules )
                if recursive:
                    rule += '+'
        elif sr == '|':
            rule += '|'
    rule += ')'
    return rule

Less than 408, more than 393

In [524]:
r = build_rule('0', rules)
sum(1 if re.fullmatch(r, m) else 0 for m in msgs) # Nope!

408

# Part 2 – Parser

In [489]:
import lark

In [490]:
with open('day19-rules.txt') as f:
    txt = f.read()
    
with open('day19-messages.txt') as f:
    msgs = f.read().splitlines()

In [491]:
grammar = """
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: \"a\"
5: \"b\"
"""

def prep_grammar( grammar ):
    grammar = re.sub("(\d+)", r"rule\1", grammar)
    return grammar

p = lark.Lark( prep_grammar(grammar), start = "rule0" )
p.parse("ababbb" )

Tree('rule0', [Tree('rule4', []), Tree('rule1', [Tree('rule3', [Tree('rule5', []), Tree('rule4', [])]), Tree('rule2', [Tree('rule5', []), Tree('rule5', [])])]), Tree('rule5', [])])

In [492]:
g = prep_grammar(txt)
p = lark.Lark(g, start = "rule0")

In [517]:
def valid( msg ):
    try:
        p.parse(msg)
        v = 1
    except Exception:
        v = 0
    
    return v

In [494]:
sum(valid(msg) for msg in msgs)

269

In [514]:
p2 = re.sub("^8:.*$", "8: 42 | 42 8", txt, flags = re.MULTILINE)
p2 = re.sub("^11:.*$", "11: 42 31 | 42 11 31", p2, flags = re.MULTILINE)

In [519]:
g = prep_grammar(p2)
p = lark.Lark(g, start = "rule0")

In [520]:
sum(valid(msg) for msg in msgs)

403